# Проведение Exploratory Data Analysis (EDA)

## 0. ВВЕДЕНИЕ

В данном ноутбуке проводится анализ даннных датасета, который содержит информацию о поражениях кожи. 
В частности, выделяется две основные группы: 
- болезни рака кожи (базальноклеточная карцинома BCC, плоскоклеточная карцинома или болезнь Боуэна SCC);
- кожные заболевания (актинический кератоз ACK, невус NEV, себорейный кератоз SEK).

Датасет имеет данные о 1373 пациентах и 26 признаков.

Для анализа будут использованы такие инструменты как:
- Pandas (статистический анализ данных)
- Matplotlib (построение графиков)
- Pandera (валидация данных)

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pandera.pandas as pa
from pandera import Column, DataFrameSchema, Check

# Добавляем настройки

pd.set_option('display.max_column', None)
# настроили отображение максимального количества столбцов
pd.set_option('display.max_rows', 100)
# настроили отображение максимального количества строк

print("Инструменты успешно импортированы и настройки проведены")

Инструменты успешно импортированы и настройки проведены
